In [ ]:
import os
import random
import warnings
import numpy as np
import pandas as pd
import pickle as pkl
import tensorflow as tf
from tensorboard.plugins import projector

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.config.threading.set_intra_op_parallelism_threads(1)


In [ ]:
from ipynb.fs.full._01_Data_Augmentation import augment

from ipynb.fs.full._02_Train_Val_Test import create_speaker_DF

from ipynb.fs.full._03_Data import create_data

from ipynb.fs.full._04_Model import do_all, do_all_train, do_all_finetune, do_all_test

from ipynb.fs.full._05_Eval import boxplot
from ipynb.fs.full._05_Eval import confusionMatrix
from ipynb.fs.full._05_Eval import time_diagramm


## Variables

In [ ]:
# general Path of all Audios
general_path = "..."
# name of folder inside general_pah for Audios
name = ''
data_path = general_path + name
# name of Augmentation ('best')
name_aug = ''
data_path_aug = general_path + name_aug
# name of folder inside general_pah for testing Audios
name_test = ''
data_path_test = general_path + name_test
# ndings of audio files for standard, dialect and testing
s_ending = 'Standard'
d_ending = 'Dialect'
t_ending = 'FreeSpeech'

# number of augmented files per original
aug_num = 6
# percentage for augmentation per file
aug_perc = 1.0
# segment length for each augmentation
aug_len = 1.0

# length of one segment
audio_length = 5.0

#model path for extracting embeddings
model_path = "...\\Models\\trillsson4"
# learning rate
lr = 0.005
# dropout rate
dr = 0.5
# units dense layer
units = 512
# size of one batch for trillsson Model
batch_size_embedding = 10
# size of one batch for CNN
batch_size = 256
# L1 regularization parameter used in the dense layers
l1 = 0.04
# L2 regularization parameter used in the dense layers
l2 = 0.035
#alpha for LeakyReLU
alpha = 0.2
# maximal number of epochs
max_epochs = 250

# number of individual runs
runs = 25
# pictures of first Epoch
first_pictures = True
# project to TB
tb = False
log_dir = '...'
# Test different Hyperparameters
hyper_test = False
# when True the Model gets trained and weights get saved
train_only = False
# dialect name for training
train_dialect = None
# when True the Model gets finetuned on previously weights and new weights get saved
finetune_only = False
# speaker label for finetuning
finetune_speaker = '' #'AM3'
# when True the Model makes predictions on Audios in 'data_path_test'
test_only = False
# speaker label for testing
test_speaker = None
# name of calibration Method to use ('temperature_scaling', 'platt', 'isotonic', 'beta' or None)
calibration = None


In [ ]:
# Function to check if a path exists
def check_valid_path(path, path_name):
    if not os.path.isdir(path):
        raise ValueError(f"Invalid path: {path_name} -> {path}")

# Check for test_only
if test_only:
    if train_only or hyper_test or tb or finetune_only:
        raise ValueError("If test_only is True, then train_only, finetune_only, hyper_test and tb must be False.")
    if name_aug != '':
        raise ValueError("If test_only is True, then name_aug must be empty.")
    if not test_speaker:
        raise ValueError("If test_only is True, then a speaker label has to be given.")
    check_valid_path(data_path_test, "data_path_test")

# Check for train_only
if train_only:
    if test_only or hyper_test:
        raise ValueError("If train_only is True, then test_only and hyper_test must be False.")
    if not train_dialect:
        raise ValueError("If train_only is True, then a dialect label has to be given.")
    check_valid_path(data_path, "data_path")
    
# Check for finetune_only
if finetune_only:
    if not finetune_speaker:
        raise ValueError("If finetune_only is True, then a speaker label has to be given.")
    if train_only or hyper_test or test_only:
        raise ValueError("If finetune_only is True, then train_only, hyper_test and test_only must be False.")

# Check for augmentation
if name_aug:
    if not (0 <= aug_perc <= 1):
        raise ValueError("aug_perc must be between 0 and 1.")
    if not (0 <= aug_len <= audio_length):
        raise ValueError("aug_len must be between 0 and audio_lengh.")
    if aug_num <= 0:
        raise ValueError("aug_num must be greater than 0.")
    if aug_perc * audio_length < aug_len:
        warnings.warn("aug_perc * audio_length is less than aug_len.", UserWarning)       

print("All checks passed successfully.")


## Augmentation

In [5]:
if (name_aug != ''):
    augment(name_aug, data_path, data_path_aug, aug_perc, aug_num, aug_len, audio_length, test)
    

## DF with all Audios

In [ ]:
df_speaker = create_speaker_DF(data_path, data_path_aug, name_aug, s_ending, d_ending, t_ending)
df_speaker = pd.read_pickle('./All_Files_.pkl')


In [ ]:
df_speaker


## Extract Embeddings

In [ ]:
if (not test_only and not finetune_only):
    df_learn = create_data(model_path, audio_length, batch_size_embedding, '', False, train_dialect)
if (name_aug != ''):
    df_learn_aug = create_data(model_path, audio_length, batch_size_embedding, name_aug, False, train_dialect)
    df_learn_aug = pd.read_pickle('./Data_' + name_aug + '_aug.pkl')
if (test_only):
    df_test = create_data(model_path, audio_length, batch_size_embedding, '', True, test_speaker)
    df_test = pd.read_pickle('./Data_test.pkl')
if (finetune_only):
    df_learn = pd.read_pickle('./Data_.pkl')
    df_finetune = df_learn[(df_learn['speaker'] == finetune_speaker) & (df_learn['class'] != 'test')]
    df_finetune.to_pickle('./Data_finetune.pkl')
    df_finetune.to_csv('./Data_finetune.csv',  sep=';')
    df_finetune = pd.read_pickle('./Data_finetune.pkl')

if (name_aug == ''):
    df_learn_aug = None

os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.config.experimental.enable_op_determinism()
    
df_learn = pd.read_pickle('./Data_.pkl')


In [ ]:
df_learn

In [10]:
if (name_aug != ''):
    print(df_learn_aug)

In [ ]:
if test_only:
    print(df_test)

In [12]:
if finetune_only:
    df_learn = pd.read_pickle('./Data_finetune.pkl')
    print(df_learn)

### Projection of embeddings to TB

In [13]:
if tb:
    labels1 = np.array(df_learn['class'].tolist())
    len_1 = len(labels1)
    labels2 = np.array(df_learn['speaker'].tolist())
    embeddings = np.array(df_learn['trillsson'].values.tolist())
    if (name_aug != ''):
            labels3 = np.array(df_learn_aug['class'].tolist())
            labels4 = np.array(df_learn_aug['speaker'].tolist())
            embeddings2 = np.array(df_learn_aug['trillsson'].values.tolist())
            labels1 = np.concatenate((labels1, labels3), axis=None)
            labels2 = np.concatenate((labels2, labels4), axis=None)
            embeddings = np.concatenate((embeddings, embeddings2), axis=0)

    with open(os.path.join(log_dir, 'metadata.tsv'), "w") as metadata:
        for i in range(0, len(labels1)):
            if i >= len_1:
                metadata.write(f'{labels1[i] + "_" + labels2[i] + "_aug"}\n')
            else:
                metadata.write(f'{labels1[i] + "_" + labels2[i]}\n')
        
    embeddings_tensor = tf.Variable(embeddings)

    checkpoint = tf.train.Checkpoint(embedding=embeddings_tensor)
    checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = 'metadata.tsv'
    projector.visualize_embeddings(log_dir, config)


## Test

In [14]:
if (not test_only):
    df_learned = pd.DataFrame(columns=['acc_train', 'acc_val', 'acc_test',
                                    'loss_train', 'loss_val', 'loss_test',
                                    'false_simplified', 'classes_x', 'classes_true', 'pred'])


In [ ]:
if train_only:
    list_row, label_mapping = do_all_train(first_pictures, df_learn, df_learn_aug, name_aug, lr, dr, units, l1, l2, alpha, batch_size, max_epochs, calibration)
    df_learned.loc[len(df_learned)] = list_row
    
elif finetune_only:
    list_row, label_mapping = do_all_finetune(first_pictures, df_learn, df_learn_aug, name_aug, lr, dr, units, l1, l2, alpha, batch_size, max_epochs)
    df_learned.loc[len(df_learned)] = list_row
        
elif test_only:
    predictions, samples_begin, samples_end = do_all_test(df_learn, df_test, lr, dr, units, l1, l2, alpha, calibration)
    predictions_df = pd.DataFrame({
        'predictions': [list(pred) for pred in predictions],
        'samples_begin': samples_begin,
        'samples_end': samples_end
    })
    predictions_df.to_csv('predictions.csv', index=False)
    predictions_df.to_pickle('predictions.pkl')

    
else:   
    for i in range(0, runs):
        print('Iteration ', i, ' of ' + str(runs))

        if (hyper_test):
            lr_test = random.uniform(lr[0], lr[1])
            dr_test = random.uniform(dr[0], dr[1])
            l1_test = random.uniform(l1[0], l1[1])
            l2_test = random.uniform(l2[0], l2[1])
            alpha_test_num = random.randint(0, len(alpha)-1)
            alpha_test = alpha[alpha_test_num]
            units_test_num = random.randint(0, len(units)-1)
            units_test = units[units_test_num]
            batch_size_test_num = random.randint(0, len(batch_size)-1)
            batch_size_test = batch_size[batch_size_test_num]
      
  
        if (hyper_test):
            list_row, label_mapping = do_all(first_pictures, df_learn, df_learn_aug, name_aug, i, lr_test, dr_test, units_test, l1_test, l2_test, alpha_test, batch_size_test, tb, log_dir, max_epochs)  
        else:
            list_row, label_mapping = do_all(first_pictures, df_learn, df_learn_aug, name_aug, i, lr, dr, units, l1, l2, alpha, batch_size, tb, log_dir, max_epochs)   
        df_learned.loc[len(df_learned)] = list_row
    

In [16]:
if (not test_only): 
    with open('label_mapping.pkl', 'wb') as f:
        pkl.dump(label_mapping, f)
    df_learned.to_pickle('./Results_' + name + '_' + name_aug + '.pkl')
    df_learned.to_csv('./Results_' + name + '_' + name_aug + '.csv',  sep=';')


## Evaluation

In [ ]:
with open('label_mapping.pkl', 'rb') as f:
    label_mapping = pkl.load(f)
print(label_mapping)

In [4]:
if (test_only):
    time_diagramm()
    

In [ ]:
if (not test_only and not train_only and not finetune_only):
    boxplot('acc_test', name, name_aug)
    confusionMatrix(name, name_aug, label_mapping)
    